In [9]:
import dgl
import torch as th

dgl的内置函数列表可参考： https://docs.dgl.ai/api/python/dgl.function.html#api-built-in

In [6]:
# 用户自定义的消息函数 等价于 dgl.function.u_add_v('hu', 'hv', 'he')
def message_func(edges):
     return {'he': edges.src['hu'] + edges.dst['hv']}
    
# 用户自定义的聚合函数 等价于 dgl.function.sum('m', 'h')
def reduce_func(nodes):
     return {'h': th.sum(nodes.mailbox['m'], dim=1)}

在DGL中，也可以在不涉及消息传递的情况下，通过 apply_edges() 单独调用逐边计算。  
apply_edges() 的参数是一个消息函数。并且在默认情况下，这个接口将更新所有的边。

In [8]:
# import dgl.function as fn

# graph.apply_edges(fn.u_add_v('el', 'er', 'e'))

对于消息传递， update_all() 是一个高级API。  
**它在单个API调用里合并了消息生成、 消息聚合和节点特征更新**，这为从整体上进行系统优化提供了空间。  
update_all() 的参数是**一个消息函数、一个聚合函数和一个更新函数**。   
更新函数是一个可选择的参数，用户也可以不使用它，而是在 update_all 执行完后直接对节点特征进行操作。   
由于更新函数通常可以用纯张量操作实现，所以DGL不推荐在 update_all 中指定更新函数。例如：

In [10]:
'''
此调用通过将源节点特征 ft 与边特征 a 相乘生成消息 m， 然后对所有消息求和来更新节点特征 ft，再将 ft 乘以2得到最终结果 final_ft。
'''

def update_all_example(graph):
    # 在graph.ndata['ft']中存储结果
    graph.update_all(fn.u_mul_e('ft', 'a', 'm'),
                     fn.sum('m', 'ft'))
    # 在update_all外调用更新函数
    final_ft = graph.ndata['ft'] * 2
    return final_ft